In [16]:
%cd ..

c:\Users\admin


## From the original drug dataset, a subset DataFrame was created focusing on two key columns: the drug name (drug_name) and the chemical composition ('chemicals'). The 'chemicals' column originally contained lists of dictionaries, which were flattened into raw text strings for further processing.

In [17]:
import pandas as pd

df = pd.read_json(r'C:\Users\admin\medical_data_saved\drugs\crawled\raw.json')
print(df.head())

                 drug_name                                               link  \
0             Parafizz 650  https://songkhoe.medplus.vn/thuoc-parafizz-650...   
1                   Damrin  https://songkhoe.medplus.vn/thuoc-damrin-lieu-...   
2                   Gonesi                https://songkhoe.medplus.vn/gonesi/   
3              Bayer (Đức)  https://www.nhathuocankhang.com/thuoc-tiet-nie...   
4  Bổ thận dương Nhất Nhất  https://songkhoe.medplus.vn/thuoc-bo-than-duon...   

                                           chemicals  \
0  Paracetamol 650 mg\n(Tá dược gồm: acid citric ...   
1               Mỗi viên nang chứa:\nDiacerein 50 mg   
2                           Mỗi viên của Gonesi chứa   
3  Trong mỗi viên Progynova 2mg chứa:\nHoạt chất:...   
4                        Mỗi viên nén bao phim chứa:   

                                               usage  \
0  \nĐiều trị các triệu chứng đau nhức và sốt từ ...   
1  Thuốc Damrin là thuốc ETC được dùng để điều...   
2  Gones

In [18]:
df['chemicals_length'] = df['chemicals'].str.len()
print(df['chemicals_length'].describe())

long_chemicals = df[df['chemicals_length'] > 2000][['drug_name', 'chemicals', 'chemicals_length']]
long_chemicals.value_counts().sum()
print(long_chemicals)

count    45088.000000
mean        84.186812
std        127.032901
min          1.000000
25%         21.000000
50%         41.000000
75%        113.000000
max       6753.000000
Name: chemicals_length, dtype: float64
                           drug_name  \
3179                Epfepara codeine   
7008                   Meyersina 100   
10300        Cao ích mẫu Mediplantex   
10329  Terpin Codein - F Mediplantex   
15619         Diclofenac 50 Cửu Long   
17855                   Noclaud 50mg   
20073                         Ukapin   
21173                  Degodas 2,5mg   
26796                  Carmotop 50mg   
29085  Paracetamol 650mg Mediplantex   
29558                    Roscef 10mg   
31051                    Zaclid 20mg   
31797                 RICHSTATIN 5mg   
32953                          ADMED   
32956                       Penveril   
33587                  Degodas 2,5mg   
33770        Cao ích mẫu Mediplantex   
34107           Medikids Mediplantex   
35279                    

In [19]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
45083     True
45084    False
45085    False
45086     True
45087    False
Length: 45088, dtype: bool

In [20]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38471 entries, 0 to 45087
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   drug_name         38471 non-null  object
 1   link              38471 non-null  object
 2   chemicals         38471 non-null  object
 3   usage             38471 non-null  object
 4   side_effects      38471 non-null  object
 5   chemicals_length  38471 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 2.1+ MB


In [21]:
sub_df = df[['drug_name', 'chemicals']].copy()
print(sub_df.head())

                 drug_name                                          chemicals
0             Parafizz 650  Paracetamol 650 mg\n(Tá dược gồm: acid citric ...
1                   Damrin               Mỗi viên nang chứa:\nDiacerein 50 mg
2                   Gonesi                           Mỗi viên của Gonesi chứa
3              Bayer (Đức)  Trong mỗi viên Progynova 2mg chứa:\nHoạt chất:...
4  Bổ thận dương Nhất Nhất                        Mỗi viên nén bao phim chứa:


In [22]:
import re
import unicodedata
import pandas as pd

def clean_chemicals(text, drug_name=None):
    if pd.isna(text):
        return pd.NA

    text = unicodedata.normalize('NFKC', str(text)).lower()

    text = re.sub(r'1\s*thành phần\s*:?', '', text)
    text = re.sub(r'(thành phần\s*:?\s*){1,2}', '', text)

    text = re.sub(r'\bmỗi\s*(viên|viên nén|viên nang|viên thuốc|lọ|ống|chai|gói|ml)?\s*(thuốc|sản phẩm)?\b', '', text)

    text = re.sub(r'(là thuốc gì.*|thuốc gì.*|cách dùng.*|thuốc này.*|câu hỏi.*|chỉ định.*|sử dụng.*)', '', text)

    if drug_name:
        drug_name_norm = unicodedata.normalize('NFKC', str(drug_name)).lower()
        text = re.sub(re.escape(drug_name_norm), '', text)

    stopwords = [
        'thành phần', 'thuốc', 'gói', 'ống', 'lọ', 'nén', 'nang',
        'bao phim', 'bao chế', 'dưới dạng', 'hàm lượng', 'được dùng', 'được',
        'bao gồm', 'là', 'chứa', 'tá dược.*', 'vừa đủ.*',
        'trong', 'sản phẩm', 'của', 'dạng', 'với'
    ]
    
    pattern_stopwords = r'\b(' + '|'.join(stopwords) + r')\b'
    text = re.sub(pattern_stopwords, '', text)

    text = re.sub(r'[^\w\s%/.,μgmgmlmcgiu]', ' ', text)

    text = re.sub(r'\s+', ' ', text).strip()

    return text

sub_df['chemicals'] = sub_df['chemicals'].apply(clean_chemicals)
sub_df['drug_name'] = sub_df['drug_name'].apply(clean_chemicals)


In [23]:
sub_df['chemicals_length'] = sub_df['chemicals'].str.len()
print(sub_df['chemicals_length'].describe())

count    38471.000000
mean        49.739596
std         87.931055
min          0.000000
25%         17.000000
50%         31.000000
75%         57.000000
max       6015.000000
Name: chemicals_length, dtype: float64


In [24]:
long_chemicals = sub_df[sub_df['chemicals_length'] > 2000][['drug_name', 'chemicals', 'chemicals_length']]
long_chemicals.value_counts().sum()
print(long_chemicals)

                     drug_name  \
3179          epfepara codeine   
7008             meyersina 100   
10300  cao ích mẫu mediplantex   
15619   diclofenac 50 cửu long   
21173            degodas 2,5mg   
26796            carmotop 50mg   
31797           richstatin 5mg   
32953                    admed   
32956                 penveril   
34107     medikids mediplantex   
35279                 niglyvid   
43052               merovia 1g   

                                               chemicals  chemicals_length  
3179   epfepara codeine hoạt chất acetaminophen 500mg...              2263  
7008   sildenafil sildenafil citrate 100mg và . bào c...              3854  
10300  cao ích mẫu mediplantex 60,43g cao lỏng hỗn hợ...              2257  
15619  diclofenac 50 cửu long hoạt chất diclofenac na...              3780  
21173  degodas 2,5mg hoạt chất ibandronic acid 2,5mg....              2412  
26796  carmotop 50mg hoạt chất metoprolol tartrat 50m...              6015  
31797  richstatin 

In [25]:
import re

def extract_chemicals(text, drug_name=None):
    if pd.isna(text) or not isinstance(text, str) or len(text.strip()) == 0:
        return [{'ingredient': '', 'dose': None}]
    
    text = text.lower()
    
    if isinstance(drug_name, str) and len(drug_name.strip()) > 0:
        drug_name = drug_name.lower()
        text = re.sub(re.escape(drug_name), '', text)
    
    units = r'mg|g|mcg|μg|ml|m\s?ui|iu|%|mg/ml|g/ml|iu/ml'
    dose_pattern = rf'(?:\d+(?:[\.,]\d+)?(?:\s*/\s*\d+(?:[\.,]\d+)?\s*)?(?:\s*(?:{units}))?)'

    matches = re.findall(
        rf'([a-zàáạảãâầấậẩẫăằắặẳẵêềếệểễèéẹẻẽôồốộổỗơờớợởỡưừứựửữùúụủũìíịỉĩỳýỵỷỹđ\s\-]+?)\s*({dose_pattern})',
        text
    )

    results = []
    for ing, dose in matches:
        ing = re.sub(r'\s+', ' ', ing.strip())
        dose = dose.replace(',', '.').replace(' ', '')
        results.append({'ingredient': ing, 'dose': dose})
    
    if not results:
        short = text.split('.')[0].split('\n')[0][:40]
        return [{'ingredient': short.strip(), 'dose': None}]
    
    return results

sub_df['chemicals'] = sub_df['chemicals'].apply(extract_chemicals)



In [26]:
sub_df.head(50)

,drug_name,chemicals,chemicals_length
0,parafizz 650,"[{'ingredient': 'paracetamol', 'dose': '650mg'}]",20
1,damrin,"[{'ingredient': 'diacerein', 'dose': '50mg'}]",15
2,gonesi,"[{'ingredient': 'gonesi', 'dose': None}]",6
3,bayer đức,"[{'ingredient': 'progynova', 'dose': '2mg'}, {...",49
4,bổ thận dương nhất nhất,"[{'ingredient': '', 'dose': None}]",0
5,propara 450mg/3ml,"[{'ingredient': 'propara', 'dose': '450mg'}]",17
6,acepron,"[{'ingredient': 'paracetamol', 'dose': None}]",11
7,newtiam,"[{'ingredient': '', 'dose': None}]",0
8,piroxicam stada 20mg,"[{'ingredient': 'piroxicam', 'dose': '20mg'}]",14
9,fuspiro,"[{'ingredient': 'fuspiro có spironolacton', 'd...",90


## Initial attempts to extract drug names from prescription images using OpenCV were unsatisfactory and will be refined later. For experimentation, a manual DataFrame was built from OCR outputs to match against a preprocessed drug list (sub_df). A fuzzy matcher, registered via the ObjectFactory class from the plasma package and based on fuzz.partial_ratio from rapidfuzz, was used under the key "fuzzy". Among the test cases, only one image yielded a partially correct match; others failed due to OCR errors or inconsistencies such as abbreviations or missing diacritics.

In [27]:
import pandas as pd

df_pics = pd.DataFrame({
    'path': [
        r"C:\Users\admin\Documents\data\.data\20231213145107_001.pdf.0000.png",
        r"C:\Users\admin\Documents\data\.data\20140119215734_001.pdf.0000.png",
        r"C:\Users\admin\Documents\data\.data\a8e7c9eca5b61ee847a7.jpg.0000.png"
    ],
    'ocr_text': [
        [
            "Ibandronic Acid (Jointmeno) 150mg – Viên uống, 1 viên/tháng, dùng buổi sáng, uống với ~150ml nước, không ăn/uống thêm trong 1h",
            "NextG Cal 500mg – Viên uống, 2 viên/trưa, 30 ngày, tổng liều 60 viên"
        ],
        [
            "Levofloxacin (LEVODHG 500) 500mg – Viên, tổng 14 viên (2 viên/ngày, sáng-tối, 7 ngày)",
            "Diosmin + hesperidin (Venokern 500mg) 450mg + 50mg – Viên, 60 viên (6 viên/ngày đầu chia 3 lần, sau uống 4 viên/ngày chia 2 lần)"
        ],
        [
            "Bình vị nam - Việt Nam – 300 viên (uống 10 viên/ngày, 5 sáng, 5 tối)",
            "Modom'S (Domperidone 10mg) – 60 viên (2 viên/ngày, sáng-tối, 1 viên/lần)",
            "Hacumin (Nano Curcumin, Royal Jelly) – 60 viên (2 viên/ngày, sáng-tối)",
            "Somastop (Sucralfat) – 60 viên (2 viên/ngày, sáng-tối)",
            "Repratt (Pantoprazol 40mg) – 60 viên (2 viên/ngày, sáng-tối)"
        ]
    ]
})

df_pics.head()

,path,ocr_text
0,C:\Users\admin\Documents\data\.data\2023121314...,[Ibandronic Acid (Jointmeno) 150mg – Viên uống...
1,C:\Users\admin\Documents\data\.data\2014011921...,"[Levofloxacin (LEVODHG 500) 500mg – Viên, tổng..."
2,C:\Users\admin\Documents\data\.data\a8e7c9eca5...,[Bình vị nam - Việt Nam – 300 viên (uống 10 vi...


In [28]:
from rapidfuzz import fuzz
from plasma.meta.class_registrator import ObjectFactory
from typing import Callable
import pandas as pd
import unicodedata
import re

matcher_factory = ObjectFactory[str, Callable[[str, pd.DataFrame], pd.Series]]()

def normalize_text(text):
    text = unicodedata.normalize('NFD', str(text))
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

def split_ocr_lines(lines: list[str]):
    split_lines = []
    units = r'(mg|ml|vien|goi|vi)'
    for line in lines:
        marked = re.sub(rf'{units}', r'\1||', line, flags=re.IGNORECASE)
        parts = marked.split('||')
        split_lines.extend([p.strip() for p in parts if len(p.strip()) > 5])
    return split_lines

sub_df['norm_name'] = sub_df['drug_name'].apply(normalize_text)
drug_keywords = set(sub_df['norm_name'].tolist())

def filter_relevant_lines(lines):
    return [l for l in lines if any(kw in normalize_text(l) for kw in drug_keywords)]

@matcher_factory.register("fuzzy")
def fuzzy_match(line: str, sub_df: pd.DataFrame):
    norm_line = normalize_text(line)
    best_score, best_row = 0, None
    for _, row in sub_df.iterrows():
        name = normalize_text(row['drug_name'])
        score = fuzz.partial_ratio(norm_line, name)
        if score > best_score and score > 92 and name in norm_line:
            best_score, best_row = score, row
    return best_row

def classify_medicine_lines(lines: list[str], sub_df: pd.DataFrame, method="fuzzy"):
    matcher = matcher_factory[method]
    results = []
    for line in lines:
        row = matcher(line, sub_df)
        if row is not None:
            results.append({
                "matched_name": row['drug_name'],
                "chemicals": row['chemicals']
            })
    return results if results else None

df_pics['ocr_lines'] = df_pics['ocr_text'].apply(split_ocr_lines)
df_pics['ocr_filtered'] = df_pics['ocr_lines'].apply(filter_relevant_lines)
df_pics['matched'] = df_pics['ocr_filtered'].apply(
    lambda lines: classify_medicine_lines(lines, sub_df, method="fuzzy")
)

for idx, row in df_pics.iterrows():
    print(f'CASE {idx + 1} | {row.get("path", "unknown")}')
    print("OCR lines:", row['ocr_lines'])
    print("Matched:", [{m['matched_name']: m['chemicals']} for m in row['matched']] if row['matched'] else "No match")

print(df_pics[['ocr_text', 'matched']].head())

registered fuzzy: <function fuzzy_match at 0x000002AA89F46340>
CASE 1 | C:\Users\admin\Documents\data\.data\20231213145107_001.pdf.0000.png
OCR lines: ['Ibandronic Acid (Jointmeno) 150mg', 'ên uống, 1 vi', 'ên/tháng, dùng buổi sáng, uống với ~150ml', 'nước, không ăn/uống thêm trong 1h', 'NextG Cal 500mg', 'ên uống, 2 vi', 'ên/trưa, 30 ngày, tổng liều 60 vi']
Matched: [{'đỏ': [{'ingredient': 'dược chất chính mecurrochrome', 'dose': None}]}, {'đỏ': [{'ingredient': 'dược chất chính mecurrochrome', 'dose': None}]}, {'đỏ': [{'ingredient': 'dược chất chính mecurrochrome', 'dose': None}]}, {'đỏ': [{'ingredient': 'dược chất chính mecurrochrome', 'dose': None}]}, {'l': [{'ingredient': 'l cystine', 'dose': '500mg'}]}, {'đỏ': [{'ingredient': 'dược chất chính mecurrochrome', 'dose': None}]}, {'đỏ': [{'ingredient': 'dược chất chính mecurrochrome', 'dose': None}]}]
CASE 2 | C:\Users\admin\Documents\data\.data\20140119215734_001.pdf.0000.png
OCR lines: ['Levofloxacin (LEVODHG 500) 500mg', 'ên, tổng 1

In [29]:
import os
import cv2
import pandas as pd
import easyocr
import re

df_pics = pd.read_pickle(r"C:\Users\admin\Documents\data\data.pkl")
base_path = r"C:\Users\admin\Documents\data\.data"
df_pics['abs_path'] = df_pics['path'].apply(lambda p: os.path.join(base_path, os.path.basename(p)))

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.resize(img, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_LINEAR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    contrast = clahe.apply(gray)
    blur = cv2.GaussianBlur(contrast, (3, 3), 0)
    thresh = cv2.adaptiveThreshold(
        blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, blockSize=25, C=10
    )
    thresh = cv2.bitwise_not(thresh)
    return thresh

reader = easyocr.Reader(['vi', 'en'])

def extract_text_from_preprocessed(img):
    if img is None:
        return []
    return reader.readtext(img, detail=0)

def fix_common_ocr_errors(text):
    text = text.lower()
    text = re.sub(r'\b1o\b', '10', text)
    text = re.sub(r'\buong\b', 'uống', text)
    text = re.sub(r'[;,_]', ' ', text)
    text = re.sub(r'[-–—]', '-', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def contains_medicine_name(text, keywords):
    text = text.lower()
    for kw in keywords:
        if kw in text:
            return True
    return False

def extract_medicine_lines(lines, keywords):
    results = []
    for line in lines:
        line_clean = line.strip()
        if len(line_clean) < 6:
            continue
        line_fixed = fix_common_ocr_errors(line_clean)

        has_unit = re.search(r'\b(\d+(mg|ml|g)|viên|gói|vỉ)\b', line_fixed)
        starts_with_num = re.match(r'^\d+\.', line_fixed)
        has_med_name = contains_medicine_name(line_fixed, keywords)

        if has_med_name or has_unit or starts_with_num:
            results.append(line_fixed)
    return results

df_pics['preprocessed'] = df_pics['abs_path'].apply(preprocess_image)
df_pics['ocr_text'] = df_pics['preprocessed'].apply(extract_text_from_preprocessed)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


c:\Users\admin\as.drug_indexer\env_py313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [30]:
print(df_pics[['path', 'ocr_text']].head())

                                      path  \
0  .data/a8e7c9eca5b61ee847a7.jpg.0000.png   
1    .data/20140119215734_001.pdf.0000.png   
2    .data/20231213145107_001.pdf.0000.png   

                                            ocr_text  
0  [BÊNII VIGN, QUÂN, s6, Khám benh ! nguY8t3 , t...  
1  [sở Y TẾHẰ NÔI, n Số & Châu Văn Ciêm Phú Đô;Na...  
2  [BỆNI VIỆN, BỆNH, ĐƠN, Số, Số, phiếu:, bệnh:, ...  
